In [ ]:
import argparse
import os
# import dicom2nifti
from skimage.measure import label, regionprops
import multiprocessing
import pandas as pd
import nibabel
# from pydicom.uid import ExplicitVRLittleEndian
import numpy as np
import sys
# import dicom2nifti.settings as settings
import SimpleITK as sitk
# import pydicom
import time
from collections import OrderedDict
import json

def write_dataset_descp(main_dir, task_name, task_id):
    task_folder_name = os.path.join(main_dir, 'nnUNet_raw', 'Task' + task_id + '_' + task_name)
    overwrite_json_file = True #make it True if you want to overwrite the dataset.json file in Task_folder
    json_file_exist = False

    if os.path.exists(os.path.join(task_folder_name,'dataset.json')):
        print('dataset.json already exist!')
        json_file_exist = True

    if json_file_exist==False or overwrite_json_file:

        json_dict = OrderedDict()
        json_dict['name'] = task_name
        json_dict['description'] = "Vessel Detection"
        json_dict['tensorImageSize'] = "3D"
        json_dict['reference'] = "see challenge website"
        json_dict['licence'] = "see challenge website"
        json_dict['release'] = "0.0"

        #you may mention more than one modality
        json_dict['modality'] = {
            "0": "MRI"
        }
        #labels+1 should be mentioned for all the labels in the dataset
        json_dict['labels'] = {
            "0": "background",
            "1": "Vessel"
        }

        train_ids = os.listdir(os.path.join(task_folder_name, 'labelsTr'))
        test_ids = os.listdir(os.path.join(task_folder_name, 'imagesTs'))

        json_dict['numTraining'] = len(train_ids)
        json_dict['numTest'] = len(test_ids)

        #no modality in train image and labels in dataset.json
        json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]
        print(json_dict['training'])

        #removing the modality from test image name to be saved in dataset.json
        json_dict['test'] = ["./imagesTs/%s" % (i[:i.find("_0000")]+'.nii.gz') for i in test_ids]
        print(json_dict['test'])

        with open(os.path.join(task_folder_name,"dataset.json"), 'w') as f:
            json.dump(json_dict, f, indent=4, sort_keys=True)

        if os.path.exists(os.path.join(task_folder_name,'dataset.json')):
            if json_file_exist==False:
                print('dataset.json created!')
            else:
                print('dataset.json overwritten!')



root = '/home/dante/nnUNet'


os.environ['nnUNet_raw'] = os.path.join(root,'nnUNet_raw')
os.environ['nnUNet_preprocessed'] = os.path.join(root,'preprocessed')
os.environ['RESULTS_FOLDER'] = os.path.join(root,'RESULTS_FOLDER')



write_dataset_descp(root, 'Vassels', '521')


In [ ]:
write_dataset_descp(root, 'TARGET', '100')

In [ ]:
# converting old version to new version

os.system('nnUNetv2_convert_old_nnUNet_dataset Task521_Vassels Dataset521_Vassels')


In [ ]:
# preprocessing

os.system('nnUNetv2_plan_and_preprocess -d 521 --verify_dataset_integrity -np 2')

In [ ]:
# training

os.system('nnUNetv2_train 521 3d_fullres 0 --npz ')

In [ ]:
# inference

os.system('nnUNetv2_predict -i input -o output -d Dataset521_Vassels -c 3d_fullres -f 0')